In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/higgs-boson/training/shard_08.tfrecord
/kaggle/input/higgs-boson/training/shard_09.tfrecord
/kaggle/input/higgs-boson/training/shard_12.tfrecord
/kaggle/input/higgs-boson/training/shard_19.tfrecord
/kaggle/input/higgs-boson/training/shard_11.tfrecord
/kaggle/input/higgs-boson/training/shard_04.tfrecord
/kaggle/input/higgs-boson/training/shard_07.tfrecord
/kaggle/input/higgs-boson/training/shard_10.tfrecord
/kaggle/input/higgs-boson/training/shard_14.tfrecord
/kaggle/input/higgs-boson/training/shard_03.tfrecord
/kaggle/input/higgs-boson/training/shard_01.tfrecord
/kaggle/input/higgs-boson/training/shard_00.tfrecord
/kaggle/input/higgs-boson/training/shard_13.tfrecord
/kaggle/input/higgs-boson/training/shard_06.tfrecord
/kaggle/input/higgs-boson/training/shard_20.tfrecord
/kaggle/input/higgs-boson/training/shard_21.tfrecord
/kaggle/input/higgs-boson/training/shard_23.tfrecord
/kaggle/input/higgs-boson/training/shard_05.tfrecord
/kaggle/input/higgs-boson/training/shard_02.tf

In [2]:
# Model Configuration
UNITS = 2 ** 11 # 2048
ACTIVATION = 'relu'
DROPOUT = 0.1

# Training Configuration
BATCH_SIZE_PER_REPLICA = 2 ** 11 # powers of 128 are best

In [3]:
# TensorFlow
import tensorflow as tf
import tensorflow_hub as hub
print("Tensorflow version " + tf.__version__)

# TF 2.3 version
# Detect and init the TPU
# try: # detect TPUs
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
#     strategy = tf.distribute.TPUStrategy(tpu)
# except ValueError: # detect GPUs
#     strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
# print("Number of accelerators: ", strategy.num_replicas_in_sync)

# TF 2.2 version
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
print("Number of accelerators: ", strategy.num_replicas_in_sync)
    
# Plotting
import pandas as pd
import matplotlib.pyplot as plt

# Matplotlib defaults
plt.style.use('seaborn-whitegrid')
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)


# Data
from kaggle_datasets import KaggleDatasets
from tensorflow.io import FixedLenFeature
AUTO = tf.data.experimental.AUTOTUNE


# Model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

Tensorflow version 2.2.0
Number of accelerators:  8


In [4]:
def make_decoder(feature_description):
    def decoder(example):
        example = tf.io.parse_single_example(example, feature_description)
        features = tf.io.parse_tensor(example['features'], tf.float32)
        features = tf.reshape(features, [28])
        label = example['label']
        return features, label
    return decoder

def load_dataset(filenames, decoder, ordered=False):
    AUTO = tf.data.experimental.AUTOTUNE
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = (
        tf.data
        .TFRecordDataset(filenames, num_parallel_reads=AUTO)
        .with_options(ignore_order)
        .map(decoder, AUTO)
    )
    return dataset

In [5]:
dataset_size = int(11e6)
validation_size = int(5e5)
training_size = dataset_size - validation_size

# For model.fit
batch_size = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
steps_per_epoch = training_size // batch_size
validation_steps = validation_size // batch_size

# For model.compile
steps_per_execution = steps_per_epoch

In [6]:
feature_description = {
    'features': FixedLenFeature([], tf.string),
    'label': FixedLenFeature([], tf.float32),
}
decoder = make_decoder(feature_description)

data_dir = KaggleDatasets().get_gcs_path('higgs-boson')
train_files = tf.io.gfile.glob(data_dir + '/training' + '/*.tfrecord')
valid_files = tf.io.gfile.glob(data_dir + '/validation' + '/*.tfrecord')

ds_train = load_dataset(train_files, decoder, ordered=False)
ds_train = (
    ds_train
    .cache()
    .repeat()
    .shuffle(2 ** 19)
    .batch(batch_size)
    .prefetch(AUTO)
)

ds_valid = load_dataset(valid_files, decoder, ordered=False)
ds_valid = (
    ds_valid
    .batch(batch_size)
    .cache()
    .prefetch(AUTO)
)
epo = 50

# Wide and Deep model (by tensorflow team)

In [ ]:
def dense_block(units, activation, dropout_rate, l1=None, l2=None):
    def make(inputs):
        x = layers.Dense(units)(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation)(x)
        x = layers.Dropout(dropout_rate)(x)
        return x
    return make


# Wide Network
wide = keras.experimental.LinearModel()

# Deep Network
inputs = keras.Input(shape=[28])
x = dense_block(UNITS, ACTIVATION, DROPOUT)(inputs)
x = dense_block(UNITS, ACTIVATION, DROPOUT)(x)
x = dense_block(UNITS, ACTIVATION, DROPOUT)(x)
x = dense_block(UNITS, ACTIVATION, DROPOUT)(x)
x = dense_block(UNITS, ACTIVATION, DROPOUT)(x)
outputs = layers.Dense(1)(x)
deep = keras.Model(inputs=inputs, outputs=outputs)

# Wide and Deep Network
wide_and_deep = keras.experimental.WideDeepModel(
    linear_model=wide,
    dnn_model=deep,
    activation='sigmoid',
)

wide_and_deep.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['AUC', 'binary_accuracy'],
#     experimental_steps_per_execution=steps_per_execution,
)

early_stopping = callbacks.EarlyStopping(
    patience=2,
    min_delta=0.001,
    restore_best_weights=True,
)

lr_schedule = callbacks.ReduceLROnPlateau(
    patience=0,
    factor=0.2,
    min_lr=0.001,
)

history = wide_and_deep.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=epo,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
)

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot(title='Cross-entropy Loss')
history_frame.loc[:, ['auc', 'val_auc']].plot(title='AUC');

# simple model with AUC instead of accuracy

In [7]:
model = keras.Sequential(
    [   
        layers.Dense(100, activation="relu", name="layer1"),
        layers.Dropout(0.2, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.Dense(150, activation="relu", name="layer2"),
        layers.Dropout(0.2, input_shape=(150,)),
        layers.BatchNormalization(),
        layers.Dense(1, name="layer3",activation='sigmoid'),
    ]
)

from keras import losses 
from keras import optimizers 
from keras import metrics 

model.compile(loss = 'binary_crossentropy',  
   optimizer = 'adam', metrics = ['AUC','binary_accuracy'])
history = model.fit(ds_train,validation_data=ds_valid, epochs = epo,steps_per_epoch=steps_per_epoch)

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss','val_loss']].plot(title='Cross-entropy Loss')
history_frame.loc[:, ['auc', 'val_auc']].plot(title='AUC');

Epoch 1/50


KeyboardInterrupt: 

# Simple Model

**try simple model with cross entropy and AUC this time**

**use your wide and deep model together putting them both in wide_and_deep.fit()

In [ ]:
model = keras.Sequential(
    [   
        layers.Dense(100, activation="relu", name="layer1"),
        layers.Dropout(0.2, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.Dense(150, activation="relu", name="layer2"),
        layers.Dropout(0.2, input_shape=(150,)),
        layers.BatchNormalization(),
        layers.Dense(1, name="layer3",activation='sigmoid'),
    ]
)

from keras import losses 
from keras import optimizers 
from keras import metrics 

model.compile(loss = 'binary_crossentropy',  
   optimizer = 'adam', metrics = ['accuracy','binary_accuracy'])
history = model.fit(ds_train,validation_data=ds_valid, epochs = epo,steps_per_epoch=steps_per_epoch)

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss','val_loss']].plot(title='Cross-entropy Loss')
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot(title='accuracy');

# Highly Dense, wide and deep Model

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(1000, activation="relu", name="layer1"),
        layers.Dropout(0.2, input_shape=(1000,)),
        layers.Dense(750, activation="relu", name="layer2"),
        layers.Dropout(0.2, input_shape=(750,)),
        layers.BatchNormalization(),
        layers.Dense(500, name="layer3",activation='relu'),
        layers.Dropout(0.2, input_shape=(500,)),
        layers.BatchNormalization(),
        layers.Dense(200, activation="relu", name="layer4"),
        layers.Dropout(0.2, input_shape=(200,)),
        layers.BatchNormalization(),
        layers.Dense(100, activation="relu", name="layer5"),
        layers.Dropout(0.2, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.Dense(50, name="layer6",activation='relu'),
        layers.Dropout(0.2, input_shape=(50,)),
        layers.BatchNormalization(),
        layers.Dense(10, activation="relu", name="layer7"),
        layers.Dropout(0.2, input_shape=(10,)),
        layers.BatchNormalization(),
        layers.Dense(1, name="layer8",activation='sigmoid'),
    ])

from keras import losses 
from keras import optimizers 
from keras import metrics 

model.compile(loss = 'binary_crossentropy',  
   optimizer = 'adam', metrics = ['accuracy','binary_accuracy'])
history = model.fit(ds_train,validation_data=ds_valid, epochs =epo ,steps_per_epoch=steps_per_epoch)

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss','val_loss']].plot(title='Cross-entropy Loss')
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot(title='accuracy');

# Wide model

In [8]:
model = keras.Sequential(
    [
        layers.Dense(100, activation="relu", name="layer1"),
        layers.Dropout(0.2, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.Dense(75, activation="relu", name="layer2"),
        layers.Dropout(0.2, input_shape=(75,)),
        layers.BatchNormalization(),
        layers.Dense(50, name="layer3",activation='relu'),
        layers.Dropout(0.2, input_shape=(50,)),
        layers.BatchNormalization(),
        layers.Dense(20, activation="relu", name="layer4"),
        layers.Dropout(0.2, input_shape=(20,)),
        layers.BatchNormalization(),
        layers.Dense(10, activation="relu", name="layer5"),
        layers.Dropout(0.2, input_shape=(10,)),
        layers.BatchNormalization(),
        layers.Dense(5, name="layer6",activation='relu'),
        layers.Dropout(0.2, input_shape=(5,)),
        layers.BatchNormalization(),
        layers.Dense(1, name="layer7",activation='sigmoid'),
    ]
)
wide=model
# from keras import losses 
# from keras import optimizers 
# from keras import metrics 

# model.compile(loss = 'binary_crossentropy',  
#    optimizer = 'adam', metrics = ['accuracy','binary_accuracy'])
# history = model.fit(ds_train,validation_data=ds_valid, epochs =epo ,steps_per_epoch=steps_per_epoch)

# history_frame = pd.DataFrame(history.history)
# history_frame.loc[:, ['loss','val_loss']].plot(title='Cross-entropy Loss')
# history_frame.loc[:, ['accuracy', 'val_accuracy']].plot(title='accuracy');

# Deep

In [9]:
model = keras.Sequential(
    [
        layers.Dense(100, activation="relu", name="layer1"),
        layers.Dropout(0.2, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.Dense(700, activation="relu", name="layer2"),
        layers.Dropout(0.2, input_shape=(700,)),
        layers.BatchNormalization(),
        layers.Dense(700, name="layer3",activation='relu'),
        layers.Dropout(0.2, input_shape=(700,)),
        layers.BatchNormalization(),
        layers.Dense(700, name="layer4",activation='relu'),
        layers.Dropout(0.2, input_shape=(700,)),
        layers.BatchNormalization(),
        layers.Dense(700, name="layer5",activation='relu'),
        layers.Dropout(0.2, input_shape=(700,)),
        layers.BatchNormalization(),
        layers.Dense(700, name="layer6",activation='relu'),
        layers.Dropout(0.2, input_shape=(700,)),
        layers.BatchNormalization(),
        layers.Dense(1, name="layer7",activation='sigmoid'),
    ]
)
deep = model

# from keras import losses 
# from keras import optimizers 
# from keras import metrics 

# model.compile(loss = 'binary_crossentropy',  
#    optimizer = 'adam', metrics = ['accuracy','binary_accuracy'])
# history = model.fit(ds_train,validation_data=ds_valid, epochs =epo ,steps_per_epoch=steps_per_epoch)

# history_frame = pd.DataFrame(history.history)
# history_frame.loc[:, ['loss','val_loss']].plot(title='Cross-entropy Loss')
# history_frame.loc[:, ['accuracy', 'val_accuracy']].plot(title='accuracy');

# wide and deep model using my two arcitectures

In [ ]:
wide_and_deep = keras.experimental.WideDeepModel(
    linear_model=wide,
    dnn_model=deep,
    activation='sigmoid',
)

wide_and_deep.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['AUC', 'binary_accuracy'],
#     experimental_steps_per_execution=steps_per_execution,
)

history = wide_and_deep.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
)

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot(title='Cross-entropy Loss')
history_frame.loc[:, ['auc', 'val_auc']].plot(title='AUC');

Epoch 1/50
640/640 [==============================] - 247s 386ms/step - loss: 0.6550 - auc: 0.6936 - binary_accuracy: 0.5300 - val_loss: 0.6194 - val_auc: 0.7517 - val_binary_accuracy: 0.5304
Epoch 2/50
640/640 [==============================] - 169s 264ms/step - loss: 0.6138 - auc: 0.7594 - binary_accuracy: 0.5300 - val_loss: 0.6017 - val_auc: 0.7760 - val_binary_accuracy: 0.5304
Epoch 3/50
640/640 [==============================] - 169s 264ms/step - loss: 0.6046 - auc: 0.7738 - binary_accuracy: 0.5305 - val_loss: 0.5951 - val_auc: 0.7907 - val_binary_accuracy: 0.5304
Epoch 4/50
640/640 [==============================] - 168s 262ms/step - loss: 0.5997 - auc: 0.7808 - binary_accuracy: 0.5316 - val_loss: 0.5909 - val_auc: 0.7966 - val_binary_accuracy: 0.5305
Epoch 5/50
640/640 [==============================] - 168s 262ms/step - loss: 0.5968 - auc: 0.7853 - binary_accuracy: 0.5341 - val_loss: 0.5889 - val_auc: 0.7998 - val_binary_accuracy: 0.5313
Epoch 6/50
640/640 [====================

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras import layers
num_classes = 5

model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/on_device_vision/classifier/landmarks_classifier_oceania_antarctica_V1/1", output_shape=[2048],
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(1, activation='softmax')
])


# model.build([None, 299, 299, 3])  # Batch input shape.


model.compile(loss = 'binary_crossentropy',  
   optimizer = 'adam', metrics = ['accuracy','binary_accuracy'])
history = model.fit(ds_train,validation_data=ds_valid, epochs =epo ,steps_per_epoch=steps_per_epoch)

In [ ]:
# import tensorflow.compat.v2 as tf
# import tensorflow_hub as hub

# m = hub.KerasLayer('https://tfhub.dev/google/on_device_vision/classifier/landmarks_classifier_oceania_antarctica_V1/1')
# m = tf.keras.Sequential([
#     m,
#     tf.keras.layers.Dense(2, activation='softmax'),
# ])


# m.compile(loss = 'binary_crossentropy',  
#    optimizer = 'adam', metrics = ['accuracy','binary_accuracy'])
# history = m.fit(ds_train,validation_data=ds_valid, epochs =12 ,steps_per_epoch=13)